In [6]:

from commonfunctions import *
from Pre_Processing_Functions import *
from skimage.transform import resize
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import cv2
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

#histogram equalization
def equalize(image):
    # Apply equalization to the image
    return cv2.equalizeHist(image)

def Histo_Equ_image(image):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return equalize(image)


def erosion(binary_image, window_size):
    height, width = binary_image.shape
    half_window = window_size // 2
    result_image = np.zeros_like(binary_image)

    for i in range(half_window, height - half_window):
        for j in range(half_window, width - half_window):
            window = binary_image[i - half_window:i + half_window + 1, j - half_window:j + half_window + 1]
            result_image[i, j] = np.min(window)

    return result_image


def dilation(binary_image, window_size):
    height, width = binary_image.shape
    half_window = window_size // 2
    result_image = np.zeros_like(binary_image)

    for i in range(half_window, height - half_window):
        for j in range(half_window, width - half_window):
            window = binary_image[i - half_window:i + half_window + 1, j - half_window:j + half_window + 1]
            result_image[i, j] = np.max(window)

    return result_image

def invert_image(image):
    # Invert the image
    return 255 - image
'''
def Pre_Processing_Func(image_path):

    # load the original image
    image = cv2.imread(image_path)

    Histo_Equ_After = Histo_Equ_image(image_path)

    histo_inverted=invert_image(Histo_Equ_After)

    histo_inverted=histo_inverted/255

    binary_image=histo_inverted > 0.7

    after_erosion=erosion(binary_image,3)

    Opened_image=dilation(after_erosion,7)

    return Opened_image
    '''
def process_image(image):
    # Perform necessary image processing operations here
    # load the original image


    Histo_Equ_After = Histo_Equ_image(image)

    histo_inverted=invert_image(Histo_Equ_After)

    histo_inverted=histo_inverted/255

    binary_image=histo_inverted > 0.7

    hist, hist_centers = histogram(binary_image)

    return hist



def detect_and_describe(img):

    img = np.array(img)[:, :, ::-1] # Convert PIL image to numpy array and reverse RGB channels (BGR -> RGB)

    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert grayscale image to 8-bit unsigned integer (np.uint8) for SIFT algorithm

    # Initialize SIFT
    sift = cv2.SIFT_create(nfeatures=200)

    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(image,None)

    return keypoints, descriptors

In [11]:
#************************************************#
#***********************MARC*********************#
#************************************************#


def hist_hsv(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Get the H channel
    image1_h = hsv_image[:, :, 0]

    # Get the S channel
    image1_s = hsv_image[:, :, 1]

    # Calculate the histograms for S, H, and V channels
    hist_h = cv2.calcHist([image1_h], [0], None, [256], [0, 256])
    hist_s = cv2.calcHist([image1_s], [0], None, [256], [0, 256])
    hist_hs=cv2.calcHist([hsv_image[:,:,:1]],[0], None, [256], [0,256])


    # Normalize the histograms
    hist_h /= hist_h.sum()
    hist_s /= hist_s.sum()
    hist_hs /= hist_hs.sum()

    return hist_h,hist_s,hist_hs


def HoG(image):
    # Resize the image to 130x276
    resized_img = cv2.resize(image, (64, 128))
    # print(resized_img.shape)

    # Convert the original image to gray scale
    resized_img_gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    # Specify the parameters for our HOG descriptor
    win_size = (64, 128)  # You need to set a proper window size
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    num_bins = 9

    # Set the parameters of the HOG descriptor using the variables defined above
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

    # Compute the HOG Descriptor for the gray scale image
    hog_descriptor = hog.compute(resized_img_gray)
    return hog_descriptor




def Sift(train_image):
  sift = cv2.SIFT_create(nfeatures=200)
  # Convert the training image to RGB
  training_image = cv2.cvtColor(train_image, cv2.COLOR_BGR2RGB)
  # Convert the training image to gray scale
  training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)
  train_keypoints, train_descriptor = sift.detectAndCompute(training_gray, None)

  return train_descriptor.flatten()

def extract_sift_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors[0].flatten() if descriptors is not None else np.array([])


def get_pixel(img, center, x, y):
  new_value = 0
  try:
    if img[x][y] >= center:
      new_value = 1
  except:
    pass
  return new_value

def lbp_calculated_pixel(img, x, y):
  center = img[x][y]
  val_ar = []
  val_ar.append(get_pixel(img, center, x-1, y-1))
  val_ar.append(get_pixel(img, center, x-1, y))
  val_ar.append(get_pixel(img, center, x-1, y + 1))
  val_ar.append(get_pixel(img, center, x, y + 1))
  val_ar.append(get_pixel(img, center, x + 1, y + 1))
  val_ar.append(get_pixel(img, center, x + 1, y))
  val_ar.append(get_pixel(img, center, x + 1, y-1))
  val_ar.append(get_pixel(img, center, x, y-1))
  power_val = [1, 2, 4, 8, 16, 32, 64, 128]
  val = 0
  for i in range(len(val_ar)):
    val += val_ar[i] * power_val[i]
  return val

def LBP_matrix(img):
  nimg = cv2.resize(img,(250,400))
  height, width, _ = nimg.shape
  img_gray = cv2.cvtColor(nimg, cv2.COLOR_BGR2GRAY)
  img_lbp = np.zeros((height, width), np.uint8)
  for i in range(0, height):
    for j in range(0, width):
      img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
  return img_lbp[0].flatten()

In [12]:
#HOG from scratch!
from pathlib import Path
from typing import Final

import numpy as np
import skimage as sk


def compute_gradient(img: np.ndarray, grad_filter: np.ndarray) -> np.ndarray:

    ts = grad_filter.shape[0]

    new_img = np.zeros((img.shape[0] + ts - 1, img.shape[1] + ts - 1))

    new_img[int((ts-1)/2.0):img.shape[0] + int((ts-1)/2.0), 
            int((ts-1)/2.0):img.shape[1] + int((ts-1)/2.0)] = img

    result = np.zeros((new_img.shape))
    
    for r in np.uint16(np.arange((ts-1)/2.0, img.shape[0]+(ts-1)/2.0)):
        for c in np.uint16(np.arange((ts-1)/2.0, img.shape[1]+(ts-1)/2.0)):
            curr_region = new_img[r-np.uint16((ts-1)/2.0):r+np.uint16((ts-1)/2.0)+1, 
                                  c-np.uint16((ts-1)/2.0):c+np.uint16((ts-1)/2.0)+1]
            curr_result = curr_region * grad_filter
            score = np.sum(curr_result)
            result[r, c] = score

    result_img = result[np.uint16((ts-1)/2.0):result.shape[0]-np.uint16((ts-1)/2.0), 
                        np.uint16((ts-1)/2.0):result.shape[1]-np.uint16((ts-1)/2.0)]

    return result_img

def compute_gradient_magnitude(horizontal_gradient: np.ndarray, vertical_gradient: np.ndarray) -> np.ndarray:

    mag_img=np.sqrt(np.power(vertical_gradient,2)+np.power(horizontal_gradient,2))
    return mag_img



def compute_gradient_direction(horizontal_gradient: np.ndarray, vertical_gradient: np.ndarray) -> np.ndarray:

    horizontal_gradient+=1e-5
    img_dir=np.rad2deg(np.arctan(vertical_gradient/horizontal_gradient))%180
    return img_dir



def find_nearest_bins(curr_direction: float, hist_bins: np.ndarray) -> (int, int):
 
    min=200
    c=0
    ind_first=10
    ind_second=10
    if (curr_direction>160):
        diff=np.abs(curr_direction-hist_bins[8])
        ind_first=8
        ind_second=0
    else:
        for i in range(9):
            new=np.floor(curr_direction // 10)
            curr_direction=new * 10
            diff=curr_direction-hist_bins[i]
            if (diff<0):
                diff=diff*-1
            if (diff<min):
                min=diff
                c=i
        ind_first=c
        ind_second=c+1

    return (ind_first,ind_second)


def update_histogram_bins(
        HOG_cell_hist: np.ndarray, 
        curr_direction: float, 
        curr_magnitude: float, 
        first_bin_idx: int, 
        second_bin_idx: int, 
        hist_bins: np.ndarray
    ) -> None:

    size=20
    ratiosecond=(hist_bins[second_bin_idx]-curr_direction)/size
    ratiofirst=1-ratiosecond
    HOG_cell_hist[first_bin_idx]+=curr_magnitude*ratiofirst
    HOG_cell_hist[second_bin_idx]+=curr_magnitude*ratiosecond


def calculate_histogram_per_cell(
        cell_direction: np.ndarray, 
        cell_magnitude: np.ndarray, 
        hist_bins: np.ndarray
    ) -> np.ndarray:


    empty_array=np.zeros(hist_bins.shape)
    
    for i in range(8):
        for j in range(8):
            first,second=find_nearest_bins(cell_direction[i,j],hist_bins)
            update_histogram_bins(empty_array,cell_direction[i,j],cell_magnitude[i,j],first,second,hist_bins)
    return empty_array


def compute_hog_features(image):

    image = sk.transform.resize(image, (128, 64))

    if image.shape[2] == 4:
        image = image[:, :, :3]

    if len(image.shape) == 3:
        image = sk.color.rgb2gray(image)

    # Define gradient masks
    horizontal_mask = np.array([-1, 0, 1])
    vertical_mask = np.array([[-1], [0], [1]])

    # Compute gradients
    horizontal_gradient = compute_gradient(image, horizontal_mask)
    vertical_gradient = compute_gradient(image, vertical_mask)

    # Compute gradient magnitude and direction
    grad_magnitude = compute_gradient_magnitude(horizontal_gradient, vertical_gradient)
    grad_direction = compute_gradient_direction(horizontal_gradient, vertical_gradient)

    # Define histogram bins
    hist_bins = np.array([10, 30, 50, 70, 90, 110, 130, 150, 170])

    cells_histogram = np.zeros((16, 8, 9))
    for r in range(0, grad_magnitude.shape[0], 8):
        for c in range(0, grad_magnitude.shape[1], 8):
            cell_direction = grad_direction[r:r+8, c:c+8]
            cell_magnitude = grad_magnitude[r:r+8, c:c+8]
            #calculate histogram per cell
           # cells_histogram[r//8, c//8] = compute_cell_histogram(cell_direction, cell_magnitude, hist_bins)

            cells_histogram[int(r/8), int(c/8)] = calculate_histogram_per_cell(cell_direction, cell_magnitude, hist_bins)

    # Normalize and concatenate histograms
    features_list = []
    for r in range(cells_histogram.shape[0] - 1):
        for c in range(cells_histogram.shape[1] - 1):
            histogram_16x16 = np.reshape(cells_histogram[r:r+2, c:c+2], (36,))
            histogram_16x16_normalized = histogram_16x16 / (np.linalg.norm(histogram_16x16) + 1e-5)
            features_list.append(histogram_16x16_normalized)

    return np.concatenate(features_list, axis=0)



In [26]:
# prepare data
input_dir = 'Data_set_33'
#categories = ['1EGP','5EGP','10EGP','20EGP','50EGP','100EGP','200EGP']
categories = ['1EGP','1SAR','5EGP','5SAR','10EGP','10SAR','20EGP','50EGP','50SAR','100EGP','100SAR','200EGP','500SAR']

data = []
feature_1 = []
feature_2 = []
feature_3 = []
feature_4 = []
feature_5 = []
feature_6 = []

labels = []

for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(input_dir, category)):
        img_path = os.path.join(input_dir, category, file)

        img = io.imread(img_path)
        # Convert the image to HSV color space
        #hist_h, hist_s, hist_hs=hist_hsv(img)

        HOG_class=compute_hog_features(img)


        #HOG_class=HoG(img)

        feature_4.append(HOG_class.flatten())

        labels.append(category_idx)


data=np.asarray(feature_4)

labels = np.asarray(labels)


In [27]:
# train / test split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

In [28]:
# train classifier
classifier = SVC()
#classifier.fit(x_train,y_train)
parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]

grid_search = GridSearchCV(classifier, parameters)

grid_search.fit(x_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001]}])

In [29]:
# test performance
best_estimator = grid_search.best_estimator_

y_prediction = best_estimator.predict(x_test)

score = accuracy_score(y_prediction, y_test)

print('{}% of samples were correctly classified'.format(str(score * 100)))

pickle.dump(best_estimator, open('./model.p', 'wb'))

78.78787878787878% of samples were correctly classified
